In [9]:
import gradio
import ollama

In [10]:
gradio.__version__

'5.21.0'

In [3]:
import ollama
ollama.api_base = "http://127.0.0.1:11434"  # Ensure correct API connection

response = ollama.chat(model='llama2', messages=[
    {'role': 'user', 
     'content': 'to harvest, is red tomato or blue tomato better?'}
],options={"num_ctx": 1024})

print(response['message']['content'])



Thank you for asking! Both red and blue tomatoes are great options for harvesting, depending on your specific needs and preferences. Here are some factors to consider:

1. Taste: Red tomatoes are generally considered to have a more intense flavor than blue tomatoes. If you want a tomato with a bold, tangy taste, red might be the better choice. On the other hand, if you prefer a milder flavor, blue tomatoes could be a good option.
2. Nutrition: Blue tomatoes are often higher in antioxidants and have a lower acidity than red tomatoes. This means they may be more nutritious overall, but this can also depend on the specific variety of each color.
3. Yield: Red tomatoes tend to produce more fruit per plant than blue tomatoes, so if you want to harvest a lot of tomatoes, red might be the better choice. However, blue tomatoes are often smaller and more compact, which can make them easier to grow in smaller spaces or for those with limited space.
4. Disease resistance: Some varieties of blue 

In [11]:
##Mirroring

import gradio as gr

def respond_message(message,chat_history):
    mirroring = message+' '+message 
    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": mirroring})

    print('my message', message)
    print('history',chat_history)

    return "", chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")   
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    msg.submit(respond_message, [msg, chatbot], [msg, chatbot])


demo.launch()


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


 *Chatbox works -----> (message,chat_history)*
-------------------------------------------------------
[-  {"role": "system", "content": "system message here"
-   {"role": "user", "content": "first user prompt here
-    {"role": "assistant", "content": "the assistant's respons
-     {"role": "user", "content": "the new user promt]},
]

*Message Sequence*
-----------------
- The system message comes first.
- Then, the history (past messages).
- Finally, the new user message.

*This is the recommended order because:*
-------------------------
- The system message sets the context for the conversation.
- The history provides prior interactions.
- The new user message continues the conversation.

In [40]:
# Integrate WITH LLM:
import gradio as gr

def greeting_message():
    return {"role": "assistant", "content": 'Hello, how can i help you?'}

def system_prompt_msg(question):
    system_prompt = "Please be kind and be straightforward in your answers. Dont eloborate the answers. "
    system_prompt += "If it is needed to give the examples especially for objective questions, add them in your comments. "
    system_prompt += "If the questions is very subjective, answer in subjective way"
    system_prompt += f"here is the question: {question}"
    return {"role": "system", "content": system_prompt}
    

def user_prompt_msg(question):
    return {"role": "user", "content": question}

def chatbot(question,msg_history):

    if not isinstance(msg_history,list): #initialize msg_history
        msg_history = []

    if not msg_history: #If it is empty by first call or clear history
        msg_history =  [greeting_message(),system_prompt_msg(question)]  
        return "",msg_history
    
    #new question comes! then save the new message coming
    msg_history.append(user_prompt_msg(question))
    
    response = ollama.chat(model='llama2', 
                       messages=msg_history,options={"num_ctx": 1024})
    
    #then save the answer
    msg_history.append({"role": "assistant", "content": response['message']['content']})
    
    #msg_history -> system_message->system_prompt->user_prompt->assistance_prompt
    #next messages in the future:
    #msg_history -> system_message->system_prompt->user_prompt->assistance_prompt->user_prompt->assistance_prompt
    print(msg_history) #for logging
  
    return '' ,msg_history
    

# Integrate the Gradio

with gr.Blocks() as demo:
    gradio_chat = gr.Chatbot(type="messages")   
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, gradio_chat])
    msg.submit(chatbot, [msg, gradio_chat], [msg, gradio_chat])

demo.launch()

* Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


**It is what exactly i want to see. The history is inside list then each message is in dict.**

But there is an observation that system message waits for user message typing. This is not my intention. Greeting message needs to be shown before user intereaction:

- The key reason is that chatbox needs to be completed the cycle includes user message and assistant message. If i do design it with streaming, it would be solved.

- It could be solved by adding value in gr.ChatBox(value ='message printing', type = 'message')

In [84]:
# how to add stream = True:

def chatbot(question,msg_history):

    if not isinstance(msg_history,list): #initialize msg_history
        msg_history = []

    if not msg_history: #If it is empty by first call or clear history
        msg_history =  [system_prompt_msg(question)]  
    
    #new question comes! then save the new message coming
    msg_history.append(user_prompt_msg(question))
    
    response = ollama.chat(model='llama2', 
                       messages=msg_history,options={"num_ctx": 1024},stream=True)
    

    cluster_of_chunk = {"role": "assistant", "content": ''}
    msg_history.append(cluster_of_chunk)
    
    for chunk in response:   # that gives the response chunk by chunk
        word = chunk.get("message",{}).get("content","")
        cluster_of_chunk['content'] += word
        yield "",msg_history
    
    print(msg_history)
    
    return '' ,msg_history
    
# Integrate the Gradio
initial_value = [{"role": "assistant", "content": "Hello, how can I help you today?"}]

with gr.Blocks() as demo:
    gradio_chat = gr.Chatbot(value=initial_value,type = "messages")   
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, gradio_chat])
    msg.submit(chatbot, [msg, gradio_chat], [msg, gradio_chat])

demo.launch()

* Running on local URL:  http://127.0.0.1:7902

To create a public link, set `share=True` in `launch()`.


[{'role': 'assistant', 'metadata': None, 'content': 'Hello, how can I help you today?', 'options': None}, {'role': 'user', 'content': 'how does the weather look like'}, {'role': 'assistant', 'content': "I'm just an AI and do not have real-time access to current weather conditions or forecasts. However, I can suggest some ways for you to find out the current weather conditions in your area or any other location:\n\n1. Check online weather websites: There are many websites that provide up-to-date weather forecasts and conditions, such as AccuWeather, Weather.com, or the National Weather Service (NWS).\n2. Use a mobile app: There are many mobile apps available that provide real-time weather information, such as Dark Sky (iOS, Android), Weather Underground (iOS, Android), or The Weather Channel (iOS, Android).\n3. Tune into local news: You can watch local news broadcasts or check their website for weather updates and forecasts.\n4. Check social media: Many meteorologists and weather organi